**RECOMMENDATION: RESTART THE NOTEBOOK FOR EACH SECTION**

---
---
---
**EXAMPLE: OPTIMIZATION OF SINGLE MONTH OPERATION FOR SINGLE BESS CONFIGURATION** (HOUR-LEVEL GRANURALITY)

In [1]:
import numpy as np

from src.utils import *
from src import *
suppress_gurobi_parallel_spam()


Set parameter Username
Set parameter LicenseID to value 2659802
Academic license - for non-commercial use only - expires 2026-05-01


In [2]:
timeseries = Timeseries(
    is_minute=False, # False -> hour-level granurality
    is_historical=True, 
    end_exclude=True, # True -> subtract one hour from given time_end
    time_start=2022090100, 
    time_end=2022100100,
)
    
parameter = Parameter(
    timeseries,
    parameter_pnnl=True, 
    pnnl_year=2023, # 2023 / 2030
    pnnl_technology="Lithium-ion_LFP", # Lithium-ion_LFP / Lithium-ion_NMC / Lead_Acid / Zinc / Vanadium_Redox_Flow
    pnnl_estimate="Point", # Point / Low / High
    pnnl_fxrate=1333, # FX rate (USDKRW) for the given period (used for OpEX & CapEX)
)

In [3]:
print_bess_config(parameter)

 [*CONFIGS FROM PNNL*]
 [**FOR GIVEN SPECS**]
 [zero  MWh   MW   hr]
 [ idx  CAP  PWR  DUR]
[[   0    2    1    2]
 [   1    4    1    4]
 [   2    6    1    6]
 [   3    8    1    8]
 [   4   10    1   10]
 [   5   20   10    2]
 [   6   24    1   24]
 [   7   40   10    4]
 [   8   60   10    6]
 [   9   80   10    8]
 [  10  100    1  100]
 [  11  100   10   10]
 [  12  200  100    2]
 [  13  240   10   24]
 [  14  400  100    4]
 [  15  600  100    6]
 [  16  800  100    8]
 [  17 1000   10  100]
 [  18 1000  100   10]]


In [4]:
# bess idx
idx_config = 3

In [5]:
solver = Solver(
    timeseries,
    parameter,
    do_single=True, # returns model if only single configuration
    idx_config=idx_config,
    do_efficiency=True, # charging and discharging efficiency
    do_rest=True, # rest before charge & rest after charge
)

solver = solver.solve()

In [ ]:
net_arbitrage_revenue = solver.model.ObjVal
print(f"{net_arbitrage_revenue:,.0f} KRW")

6,855,398 KRW


---
---
---
**EXAMPLE: OPTIMIZATION OF 12 MONTHS OPERATION FOR SINGLE BESS CONFIGURATION** (HOUR-LEVEL GRANURALITY)

In [7]:
import numpy as np

from src.utils import *
from src import *
suppress_gurobi_parallel_spam()

In [8]:
# bess idx
idx_config = 12

In [9]:
# replaced by bash commands (capitalized)
DATE = np.concatenate((np.arange(0, 120000, 10000) + 2022010100, np.array([2023010100])))
ITERATOR = enumerate(zip(DATE[:-1].tolist(), DATE[1:].tolist()))

In [10]:
TOTAL_NET_ARBITRAGE_REVENUE = 0

for IDX_MONTH, (TIME_START, TIME_END) in ITERATOR:

    timeseries = Timeseries(
        is_minute=False, 
        is_historical=True, 
        end_exclude=True, 
        time_start=TIME_START, 
        time_end=TIME_END,
    )
        
    parameter = Parameter(
        timeseries,
        parameter_pnnl=True, 
        pnnl_year=2023,
        pnnl_technology="Lithium-ion_LFP",
        pnnl_estimate="Point",
        pnnl_fxrate=1333, # advised to keep the rate identical for all months for simpler CapEX calculation
    )

    solver = Solver(
        timeseries,
        parameter,
        do_single=False, # returns model if only single configuration
        idx_config=idx_config,
        do_efficiency=True,
        do_rest=True,
    )

    TOTAL_NET_ARBITRAGE_REVENUE += solver.solve()

In [11]:
print(f"{TOTAL_NET_ARBITRAGE_REVENUE:,.0f} KRW")

1,775,336,819 KRW
